<a href="https://colab.research.google.com/github/pvipin08/Trading_PivotPoints/blob/main/Copy_of_Yearly_Fibonacci_pivot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import pandas as pd

path = "/content/drive/MyDrive/ind-market-dataset/ind_niftytotalmarket_list.csv"
#path = "/content/drive/MyDrive/ind-market-dataset/ind_niftytotalmarket_list_test.xlsx"

df=pd.read_csv(path)
#df = pd.read_excel(path)
df.head(5)

,Company Name,Industry,Symbol,Series,ISIN Code
0,360 ONE WAM Ltd.,Financial Services,360ONE,EQ,INE466L01038
1,3M India Ltd.,Diversified,3MINDIA,EQ,INE470A01017
2,ABB India Ltd.,Capital Goods,ABB,EQ,INE117A01022
3,ACC Ltd.,Construction Materials,ACC,EQ,INE012A01025
4,ACME Solar Holdings Ltd.,Power,ACMESOLAR,EQ,INE622W01025


In [ ]:
# Install necessary packages
!pip install yfinance pandas tqdm datetime --quiet

import pandas as pd
import yfinance as yf
#from datetime import datetime
from datetime import date
from tqdm.notebook import tqdm

# STEP 1: Read your Nifty stock symbols from CSV
# Make sure your CSV has a column named 'Symbol'
nifty_df = df
nifty_symbols = nifty_df['Symbol'].tolist()

# STEP 2: Set up year selection
today = date.today()
current_year = today.year
last_year = current_year - 1
start_date = f"{last_year}-01-01"
end_date = f"{last_year}-12-31"

print(today, current_year, last_year, start_date, end_date)

def fibonacci_pivots(h, l, c):
    pp = (h + l + c) / 3
    return {
        'PP': float(pp),
        'R1': float(pp + (h - l) * 0.382),
        'R2': float(pp + (h - l) * 0.618),
        'R3': float(pp + (h - l)),
        'S1': float(pp - (h - l) * 0.382),
        'S2': float(pp - (h - l) * 0.618),
        'S3': float(pp - (h - l))
    }

below_S2 = []
below_S3 = []
below_PP = []
below_R3 = []

# STEP 3: For each stock, download historical data and apply Fibonacci filter
for symbol in tqdm(nifty_symbols):
    ticker = symbol + ".NS"
    try:
        # Download ALL data since last year's Jan 1
        df = yf.download(ticker, start=start_date, end=today, interval = '1d', progress=False);
        if df.empty or (len(df[df.index.year == last_year]) == 0) or (len(df[df.index.year == current_year]) == 0):
            continue

        # Get last year's yearly OHLC
        df_last_year = df[df.index.year == last_year]
        print(df_last_year)
        o = df_last_year.iloc[0]['Open']         # Opening price first trading day
        h = df_last_year['High'].max()           # High of last year
        l = df_last_year['Low'].min()            # Low of last year
        c = df_last_year.iloc[-1]['Close']       # Close of last trading day of Dec
        print(o,h,l,c)

        pivots = fibonacci_pivots(h, l, c)

        # Get most recent close for this year
        df_this_year = df[df.index.year == current_year]
        current_close = float(df_this_year.iloc[-1]['Close'])
        print (f"PP:{pivots['PP']}, S2:{pivots['S2']}, S3:{pivots['S3']}")

        if current_close < pivots['S2']:
            below_S2.append(symbol)
        if current_close < pivots['S3']:
            below_S3.append(symbol)
        if current_close < pivots['PP']:
            below_PP.append(symbol)
        if current_close < pivots['R3']:
            below_R3.append(symbol)

    except Exception as e:
        print(f"Error for {symbol}: {str(e)}")
        continue


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.4/264.4 kB 9.5 MB/s eta 0:00:00
2025-09-13 2025 2024 2024-01-01 2024-12-31


  0%|          | 0/752 [00:00<?, ?it/s]

/tmp/ipython-input-1126179982.py:46: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=today, interval = '1d', progress=False);


Price             Close         High          Low         Open    Volume
Ticker        360ONE.NS    360ONE.NS    360ONE.NS    360ONE.NS 360ONE.NS
Date                                                                    
2024-01-01   688.743286   700.061517   679.091006   695.602791    175009
2024-01-02   663.314209   685.901569   661.452319   681.295889    348865
2024-01-03   642.098694   668.360788   639.795884   667.429843    203415
2024-01-04   640.726807   648.762240   638.179014   646.753396    133259
2024-01-05   649.154236   651.604059   631.123511   640.726807   1231143
...                 ...          ...          ...          ...       ...
2024-12-24  1228.810547  1232.140094  1193.874872  1222.499294    284990
2024-12-26  1248.589233  1304.098629  1206.845355  1228.860305   1129000
2024-12-27  1243.271851  1269.162921  1214.448697  1255.148929    827586
2024-12-30  1263.199463  1287.102850  1214.299542  1247.297034    711413
2024-12-31  1247.148071  1267.622425  1235.419981  

/tmp/ipython-input-1126179982.py:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  'PP': float(pp),
/tmp/ipython-input-1126179982.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  'R1': float(pp + (h - l) * 0.382),
/tmp/ipython-input-1126179982.py:29: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  'R2': float(pp + (h - l) * 0.618),
/tmp/ipython-input-1126179982.py:30: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  'R3': float(pp + (h - l)),
/tmp/ipython-input-1126179982.py:31: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float

...

/tmp/ipython-input-1126179982.py:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  'PP': float(pp),
/tmp/ipython-input-1126179982.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  'R1': float(pp + (h - l) * 0.382),
/tmp/ipython-input-1126179982.py:29: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  'R2': float(pp + (h - l) * 0.618),
/tmp/ipython-input-1126179982.py:30: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  'R3': float(pp + (h - l)),
/tmp/ipython-input-1126179982.py:31: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float

Price             Close         High          Low         Open       Volume
Ticker     ZYDUSWELL.NS ZYDUSWELL.NS ZYDUSWELL.NS ZYDUSWELL.NS ZYDUSWELL.NS
Date                                                                       
2024-01-01  1672.072266  1690.971357  1651.183796  1684.107979        47546
2024-01-02  1664.811035  1680.079609  1641.236953  1671.077625        61230
2024-01-03  1680.577026  1684.704889  1653.173293  1664.811107        40503
2024-01-04  1677.393921  1688.982073  1657.400694  1685.998006        38748
2024-01-05  1682.466797  1688.982059  1666.651338  1672.122128        32536
...                 ...          ...          ...          ...          ...
2024-12-24  1904.009521  1907.299737  1876.392109  1876.392109         8044
2024-12-26  1893.989502  1923.202122  1888.705246  1916.422457        19356
2024-12-27  1892.892822  1901.018440  1869.412947  1892.444090       107911
2024-12-30  1890.948608  1900.320623  1856.501631  1881.626493        21102
2024-12-31  

/tmp/ipython-input-1126179982.py:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  'PP': float(pp),
/tmp/ipython-input-1126179982.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  'R1': float(pp + (h - l) * 0.382),
/tmp/ipython-input-1126179982.py:29: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  'R2': float(pp + (h - l) * 0.618),
/tmp/ipython-input-1126179982.py:30: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  'R3': float(pp + (h - l)),
/tmp/ipython-input-1126179982.py:31: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float

Price             Close         High          Low         Open    Volume
Ticker        ECLERX.NS    ECLERX.NS    ECLERX.NS    ECLERX.NS ECLERX.NS
Date                                                                    
2024-01-01  2552.432373  2633.434261  2538.490711  2618.892858     73500
2024-01-02  2503.811523  2586.462338  2482.574149  2563.476004    113297
2024-01-03  2581.515137  2597.855275  2503.511467  2517.453129     65612
2024-01-04  2553.731445  2588.510813  2533.493480  2588.460794     45252
2024-01-05  2571.021484  2588.461067  2512.756076  2563.226070     60790
...                 ...          ...          ...          ...       ...
2024-12-24  3624.843262  3667.082696  3522.118958  3667.082696     73836
2024-12-26  3551.111572  3628.642224  3501.973963  3617.195137     61044
2024-12-27  3649.187256  3661.684130  3530.316845  3548.862254     69551
2024-12-30  3501.523926  3648.237223  3439.189666  3629.092110    158016
2024-12-31  3482.528809  3517.170290  3452.186545  

/tmp/ipython-input-1126179982.py:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  'PP': float(pp),
/tmp/ipython-input-1126179982.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  'R1': float(pp + (h - l) * 0.382),
/tmp/ipython-input-1126179982.py:29: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  'R2': float(pp + (h - l) * 0.618),
/tmp/ipython-input-1126179982.py:30: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  'R3': float(pp + (h - l)),
/tmp/ipython-input-1126179982.py:31: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float

In [ ]:
print("Stocks currently below S2:", below_S2)
print("Stocks currently below S3:", below_S3)
print("Stocks currently below PP:", below_PP)

Stocks currently below S2: ['ABFRL', 'ALLCARGO', 'BSOFT', 'FIRSTCRY', 'CEIGALL', 'CELLO', 'CENTRALBK', 'CLEAN', 'CONCOR', 'CYIENTDLM', 'CYIENT', 'DEEPAKNTR', 'EASEMYTRIP', 'EMBDL', 'FCL', 'FIVESTAR', 'GATEWAY', 'HAPPSTMNDS', 'IIFLCAPS', 'INDIGOPNTS', 'INFIBEAM', 'IGIL', 'IKS', 'JAIBALAJI', 'JAICORPLTD', 'JINDWORLD', 'KNRCON', 'MAHLIFE', 'MANINFRA', 'NOCIL', 'NTPCGREEN', 'ORCHPHARMA', 'PATELENG', 'PRAJIND', 'RAYMONDLSL', 'RAYMOND', 'RESPONIND', 'ROUTE', 'SHAREINDIA', 'RENUKA', 'SIEMENS', 'SONACOMS', 'SONATSOFTW', 'SWSOLAR', 'SULA', 'TVSSCS', 'TCS', 'TATATECH', 'TEAMLEASE', 'UCOBANK', 'UNIMECH', 'VBL', 'MANYAVAR']
Stocks currently below S3: ['ABFRL', 'FIRSTCRY', 'CYIENT', 'IGIL', 'IKS', 'JINDWORLD', 'ROUTE', 'SHAREINDIA', 'SONACOMS', 'TEAMLEASE', 'UNIMECH']
Stocks currently below PP: ['3MINDIA', 'ABB', 'ACC', 'AGI', 'AIAENG', 'AWL', 'AARTIIND', 'AAVAS', 'ACE', 'ADANIENSOL', 'ADANIENT', 'ADANIGREEN', 'ATGL', 'ABFRL', 'ABREL', 'ADVENZYMES', 'AEGISLOG', 'AETHER', 'AFCONS', 'AHLUCONT', 'AJAN